In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pykrx import stock
from datetime import datetime, timedelta

# 1️⃣ 종목 코드 가져오기 함수
def get_top_n_codes(sosok=0, n=20):
    url = f"https://finance.naver.com/sise/sise_market_sum.naver?sosok={sosok}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    response.encoding = 'euc-kr'
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='type_2')
    rows = table.find_all('tr')[2:]  # 상단 2줄은 헤더이므로 제외
    data = []

    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 1:
            name = cols[1].text.strip()
            code = cols[1].find('a')['href'].split('=')[-1]
            data.append((name, code))

    df_top_n = pd.DataFrame(data, columns=['Name', 'Code']).head(n)
    return df_top_n

# 2️⃣ KOSPI 및 KOSDAQ 상위 종목 코드 가져오기
df_kospi_top20 = get_top_n_codes(sosok=0, n=20)
df_kosdaq_top20 = get_top_n_codes(sosok=1, n=20)
df_top_40 = pd.concat([df_kospi_top20, df_kosdaq_top20], ignore_index=True)
codes_list = df_top_40['Code'].tolist()

# 3️⃣ 기간 설정
periods = {
    "1주일": timedelta(weeks=1),
    "2주일": timedelta(weeks=2),
    "1개월": timedelta(days=30),
    "3개월": timedelta(days=90),
    "6개월": timedelta(days=180),
    "12개월": timedelta(days=365),
}

# 4️⃣ 외국인 순매수 상위 종목 추출 함수
def get_top_foreign_net_buy(period_name, start_date, end_date):
    results = []
    for code in codes_list:
        try:
            # 외국인 거래대금 데이터 가져오기
            value_data = stock.get_market_trading_value_by_investor(start_date, end_date, code)
            foreign_value = value_data.loc["외국인"]["순매수"]
            total_net_buy = foreign_value.sum()
            results.append((code, total_net_buy))
        except:
            continue

    # 거래대금 순매수 상위 10개 종목 정렬
    sorted_results = sorted(results, key=lambda x: x[1], reverse=True)[:10]
    return sorted_results

# 5️⃣ 각 기간별 상위 종목 출력
today = datetime.now()
for period_name, delta in periods.items():
    start_date = (today - delta).strftime("%Y%m%d")
    end_date = today.strftime("%Y%m%d")
    top_stocks = get_top_foreign_net_buy(period_name, start_date, end_date)
    print(f"\n📊 {period_name} 동안 외국인 순매수 상위 종목:")
    for i, (code, net_buy) in enumerate(top_stocks, 1):
        print(f"{i}. 종목 코드: {code}, 순매수 금액: {net_buy:,.0f}원")


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from pykrx import stock
import matplotlib.pyplot as plt

# ✅ 1️⃣ 종목 코드 가져오기 함수
def get_top_n_codes(sosok=0, n=20):
    url = f"https://finance.naver.com/sise/sise_market_sum.naver?sosok={sosok}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    response.encoding = 'euc-kr'
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='type_2')
    rows = table.find_all('tr')[2:]
    data = []

    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 1:
            name = cols[1].text.strip()
            code = cols[1].find('a')['href'].split('=')[-1]
            data.append((name, code))

    df_top_n = pd.DataFrame(data, columns=['Name', 'Code']).head(n)
    return df_top_n

# ✅ 2️⃣ KOSPI 및 KOSDAQ 상위 종목 코드 가져오기
df_kospi_top20 = get_top_n_codes(sosok=0, n=20)
df_kosdaq_top20 = get_top_n_codes(sosok=1, n=20)
df_top_40 = pd.concat([df_kospi_top20, df_kosdaq_top20], ignore_index=True)

# 종목 코드-이름 매핑 딕셔너리 생성
code_name_mapping = dict(zip(df_top_40['Code'], df_top_40['Name']))

# ✅ 3️⃣ 기간 설정
periods = {
    "1주일": timedelta(weeks=1),
    "2주일": timedelta(weeks=2),
    "1개월": timedelta(days=30),
    "3개월": timedelta(days=90),
    "6개월": timedelta(days=180),
    "12개월": timedelta(days=365),
}

# ✅ 4️⃣ 외국인 순매수 상위 종목 추출 함수
def get_top_foreign_net_buy(start_date, end_date):
    results = []
    for code, name in code_name_mapping.items():
        try:
            value_data = stock.get_market_trading_value_by_investor(start_date, end_date, code)
            foreign_value = value_data.loc["외국인"]["순매수"].sum()
            results.append((name, foreign_value))
        except Exception:
            continue
    sorted_results = sorted(results, key=lambda x: x[1], reverse=True)[:10]
    return sorted_results


# ✅ 차트에서 X축을 억 원 단위로 변환하고, 캔들 옆에 금액을 표시
today = datetime.now()
for period_name, delta in periods.items():
    start_date = (today - delta).strftime("%Y%m%d")
    end_date = today.strftime("%Y%m%d")
    top_stocks = get_top_foreign_net_buy(start_date, end_date)

    # 그래프 데이터 준비
    names = [x[0] for x in top_stocks]
    values = [x[1] / 1e8 for x in top_stocks]  # 억 원 단위로 변환

    # 그래프 생성
    plt.figure(figsize=(10, 6))
    bars = plt.barh(names, values, color='skyblue')
    plt.title(f'{period_name} 동안 외국인 순매수 상위 종목')
    plt.xlabel('순매수 금액 (억 원)')
    plt.tight_layout()
    plt.gca().invert_yaxis()

    # 금액을 캔들 옆에 표시
    for bar, value in zip(bars, values):
        if value >= 10000:
            text = f'{value / 10000:.1f}조원'
        else:
            text = f'{value:.0f}억원'
        plt.text(value + 100, bar.get_y() + bar.get_height() / 2, text, va='center', fontsize=10)

    plt.show()
